### Задание

Самостоятельно напишите нейронную сеть, которая может стать составной частью системы бота для игры в "Крестики-нолики". Используя подготовленную базу изображений, создайте и обучите нейронную сеть, распознающую две категории изображений: крестики и нолики. Добейтесь точности распознавания более 95% (accuracy)


# Импорт необходимых библиотек и модулей
В этом блоке происходит импорт всех нужных компонентов для создания и обучения нейронной сети, распознающей изображения крестиков и ноликов.
Импорт включает:
- Sequential — создание последовательной нейронной сети (один слой за другим).

- Dense, Dropout — полносвязные слои и слой регуляризации Dropout.

- Adam — оптимизатор, устойчивый к шуму и хорошо подходящий для большинства задач.

- utils — для функции to_categorical, преобразующей метки в one-hot формат.

- image — для загрузки и предварительной обработки изображений.

- numpy — массивы и математика.

- os — для работы с файловой системой.

- matplotlib.pyplot — визуализация изображений и графиков.

- PIL.Image — дополнительная обработка изображений (если нужно работать напрямую с пикселями).

- warnings — подавление несущественных предупреждений.

- %matplotlib inline — директива Jupyter для отображения изображений прямо в ноутбуке.

In [1]:
# Стандартная библиотека
import os
import warnings
import zipfile  # работа с zip-архивами

# Сторонние библиотеки
import gdown  # загрузка файлов по HTML ссылке
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# TensorFlow / Keras
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Подавление предупреждений
warnings.filterwarnings('ignore')

# Вывод изображений в Jupyter-ноутбуке
%matplotlib inline

#  Загрузка и распаковка датасета
Этот блок отвечает за автоматическую загрузку архива с изображениями, содержащими крестики и нолики, а также его распаковку:

gdown.download(...) — загружает архив hw_pro.zip, содержащий изображения двух классов.


In [2]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_pro.zip', None, quiet=True)

'hw_pro.zip'

**Распаковка архива**

Код открывает ZIP-файл hw_pro.zip для чтения и извлекает все его содержимое в папку hw_pro.
Если папка hw_pro не существует, она будет создана автоматически.



In [3]:
with zipfile.ZipFile('hw_pro.zip', 'r') as zip_ref:
    zip_ref.extractall('hw_pro')

# Загрузка и разметка изображений из датасета "крестики-нолики"
В этом блоке происходит загрузка изображений из папок с двумя классами и формирование обучающего набора:

- base_dir = '/content/hw_pro' — путь к директории с распакованным датасетом.

- x_train — список для хранения изображений.

- y_train — список для хранения меток классов (0 или 1).

- img_height и img_width — размеры изображений, к которым приводятся все картинки (20×20 пикселей).

Логика обхода:
Проход по папкам в директории базы данных, каждая папка — отдельный класс:

- Папка '0' — метка 0 (например, крестики),

- Все остальные — метка 1 (например, нолики).

Для каждого изображения:

- Загружается с приведением к размеру 20×20 и в оттенках серого.

- Преобразуется в числовой массив.

- Добавляется в список x_train.

Метка класса добавляется в y_train.


Далее списки преобразуются в NumPy-массивы для удобства обработки в дальнейшем. Выводятся размеры массивов.

In [ ]:
# Путь к директории с базой
base_dir = '/content/hw_pro'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                         target_size=(img_height, img_width),
                                                         color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        else:
            y_train.append(1)
# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (102, 20, 20, 1)
Размер массива y_train (102,)


# Преобразование формы входных данных
В этом блоке происходит изменение формы массива изображений для подачи в полносвязный слой нейронной сети:

- Исходные изображения имеют форму (число_образцов, 20, 20, 1).

- Метод reshape преобразует каждое изображение в вектор длиной 400 (20×20), сохраняя размер батча (x_train.shape[0]).

- Итоговая форма данных будет (число_образцов, 400) — это требуется для подачи в полносвязный слой, который принимает одномерный вход.

- Для проверки выводится новая форма массива x_train.

In [ ]:
# Изменение формы входных картинок с 20х20 на 400
# первая ось остается без изменения, остальные складываются в вектор
x_train = x_train.reshape(x_train.shape[0], -1)

# Проверка результата
print(f'Форма обучающих данных: {x_train.shape}')

Форма обучающих данных: (102, 400)


## Нормализация изображений
- Преобразуем тип данных массива изображений в float32.

- Нормализуем значения пикселей, разделив их на 255, чтобы привести их к диапазону [0, 1].

In [ ]:
x_train = x_train.astype('float32') / 255.

В этом блоке метки классов (0 и 1) преобразуются в формат one-hot векторов, чтобы подходить под многоклассовую задачу классификации:

- Используется функция to_categorical из tensorflow.keras.utils.

- Количество классов — 2 (крестики и нолики).

- Пример: метка 0 преобразуется в [1, 0], а 1 — в [0, 1].

In [ ]:
# Преобразование ответов в формат one_hot_encoding
y_train = utils.to_categorical(y_train, 2)

# Создание архитектуры нейронной сети
В этом блоке формируется последовательная модель нейронной сети для классификации изображений крестиков и ноликов:

Первый слой:

- Dense(100, input_dim=400, activation='relu')

- Полносвязный слой с 100 нейронами,

- Принимает вход размерности 400 (развернутое изображение 20×20),

- Использует функцию активации ReLU для нелинейности.

Второй слой:

- Dense(50, activation='relu')

- Полносвязный слой с 50 нейронами и ReLU-активацией.

Выходной слой:

- Dense(2, activation='softmax')

- Выход из 2 нейронов — по количеству классов (крестики и нолики),

- Softmax преобразует выходы в вероятности принадлежности к каждому классу.



In [ ]:
# Создание последовательной модели
model = Sequential()

model.add(Dense(100, input_dim=400, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))


Компиляция и просмотр структуры модели.
model.compile(...) — конфигурация модели:

- loss='categorical_crossentropy' — функция потерь для многоклассовой классификации (подходит для one-hot меток),

- optimizer='adam' — оптимизатор Adam, один из самых эффективных для большинства задач,

- metrics=['accuracy'] — метрика, по которой будет оцениваться качество работы модели (точность).

model.summary() — выводит краткое описание модели.

In [ ]:
# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 100)            │        40,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 2)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,252 (176.77 KB)

 Trainable params: 45,252 (176.77 KB)

 Non-trainable params: 0 (0.00 B)

None


# Обучение нейронной сети
Этот блок запускает процесс обучения модели с использованием подготовленных входных и выходных данных:

- model.fit(...) — основная функция обучения модели.

- x_train, y_train — обучающая выборка (входы и метки).

- batch_size=10 — количество примеров, обрабатываемых перед одним обновлением весов (чем меньше, тем медленнее обучение, но может быть устойчивее).

- epochs=10 — количество полных проходов по всему датасету.

- verbose=1 — вывод процесса обучения (значения loss и accuracy по эпохам).

In [ ]:
# Обучение модели с сохранением истории
history = model.fit(x_train,
                    y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=1)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0717 
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0645  
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0411 
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0400 
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0307
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0209
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0193  
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0199 
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0151 
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0135 


In [ ]:
# Вывод точности на последней эпохе
print(f"Финальная точность: {history.history['accuracy'][-1]}")

Финальная точность: 1.0


# Вывод по работе
В ходе выполнения задания была успешно создана и обучена нейронная сеть для распознавания изображений крестиков и ноликов. Использовалась база изображений с двумя категориями, которая была предварительно подготовлена и нормализована. Модель состояла из двух скрытых слоёв с активацией ReLU и выходного слоя с функцией softmax. После обучения на протяжении 10 эпох с небольшим размером батча сеть достигла точности 100% на обучающей выборке, что существенно превышает требуемый порог в 95%. Это свидетельствует о том, что модель эффективно научилась различать изображения двух классов и полностью справляется с поставленной задачей.